In [1]:
import numpy as np

import torch, torch.nn as nn, torch.autograd as autograd
import torch.utils.data
from torch.distributions import multivariate_normal

In [2]:
!deepwalk --input ../graphsage/cora/cora.adjlist --representation-size 512 --walk-length 40 --output ../graphsage/cora/cora.embeddings

Number of nodes: 2708
Number of walks: 27080
Data size (walks*length): 1083200
Walking...
Training...
/usr/local/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
def vectorize(row):
    return np.array(row.split(" ")).astype(float)

In [4]:
with open("../graphsage/cora/cora.embeddings", "r") as f:
    skipgram = f.readlines()
    
print("Number of nodes: {} in {}-dimensional representation".format(*skipgram.pop(0).split()))

dataset = {}
for i in range(len(skipgram)):
    id, v = skipgram[i].split(" ", 1)
    dataset[int(id)] = vectorize(v)

Number of nodes: 2708 in 512-dimensional representation


In [62]:
model = nn.Sequential(
    nn.Linear(512, 256),
    nn.Dropout(0.05),
    nn.Linear(256, 128),
    nn.Dropout(0.05),
    nn.Linear(128, 64),
    nn.Dropout(0.05),
    nn.ReLU(),
    nn.Linear(64, 7),
    nn.Softmax()
)

In [6]:
batch_size = 400
train_batch_gen = torch.utils.data.DataLoader(dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=1)

train_batch_gen

In [60]:
def compute_loss(model, X_batch, psi, Sigma):
    # Output (X_batch, 7)
    logits            = model(X_batch)
    print("Logits shape: {}".format(logits.shape))
    print(logits[0].sum())
    print(logits[:, 0].sum())
    # Probabilities (X_batch, 7)
    probs             = compute_probs(X_batch, psi, Sigma)
    # Gamma
    gamma_numerator   = logits * probs.exp()
    gamma_denominator = torch.sum(logits * probs.exp(), dim=1, keepdim=True)
    gamma             = gamma_numerator / gamma_denominator
    print("Gamma shape: {}".format(gamma.shape))
    # N
    N                 = gamma.sum(dim=0, keepdim=True)
    print("N shape: {}".format(N.shape))
    print(N)
    # Pi
    pi                = N / X_batch.size(0)
    print("Pi shape: {}".format(pi.shape))
    print(pi)
    # Psi (mean)
    psi               = torch.FloatTensor()
    # Sigma (covariance)
    Sigma             = torch.FloatTensor()
    for k in range(7):
        psi_k   = compute_mean_k(N[:, k], gamma[:, k], X_batch)
        Sigma_k = compute_cov_k(N[:, k], gamma[:, k], X_batch, psi_k)
        psi     = torch.cat((psi, psi_k), dim=0)
        Sigma   = torch.cat((Sigma, Sigma_k), dim=0)
    print("Psi shape: {}".format(psi.shape))
    print("Sigma shape: {}".format(Sigma.shape))
    
    # return -1 / 7 * 
    
def compute_mean_k(N_k, gamma_k, phi):
    return (1 / N_k) * torch.sum(gamma_k.view(-1, 1) * phi, dim=0, keepdim=True)

def compute_cov_k(N_k, gamma_k, phi, psi_k):
    return (1 / N_k) * torch.sum(gamma_k.view(-1, 1) * (phi - psi_k) * (phi - psi_k), dim=0, keepdim=True)
    
def compute_probs(X_batch, psi, Sigma):
    probs = torch.FloatTensor()
    
    for psi_k, Sigma_k in list(zip(psi, Sigma)):
        # Multivariate Normal       
        normal_k = multivariate_normal.MultivariateNormal(psi_k, Sigma_k)
        
        # Probability (X_batch, 1)
        prob_k   = normal_k.log_prob(X_batch).view(X_batch.size()[0], 1)
        probs    = torch.cat((probs, prob_k), dim=1)
        
    return probs

In [61]:
torch.manual_seed(1234)

psi = []
Sigma = []

opt = torch.optim.SGD(model.parameters(), lr=0.1)

num_epochs = 10
train_loss = []

# Initialize Multivariate Gaussian distributions
train_batch_gen_iter = iter(train_batch_gen)
for i in range(7):
    X_batch = next(train_batch_gen_iter)
    data    = X_batch.float().numpy()
    psi_k   = np.mean(data, axis=0)
    Sigma_k = np.cov(data, rowvar=0)
    # Ensure that the covariance matrix is positive definite
    Sigma_k = np.add(Sigma_k, np.multiply(np.eye(512), 1/7))
    # Add k-th distribution to the list
    psi.append(psi_k)
    Sigma.append(Sigma_k)
    # print("Psi shape: {}".format(psi_k.shape))
    # print("Sigma shape: {}".format(Sigma_k.shape))

for epoch in range(num_epochs):
    model.train(True)
    
    print("Epoch: {}".format(epoch))
    
    for X_batch in train_batch_gen:    
        with autograd.detect_anomaly():
            # Update Sigma
            loss = compute_loss(model, X_batch.float(), torch.tensor(psi).float(), torch.tensor(Sigma).float())
            loss.backward()
            
            opt.step()
            opt.zero_grad()
            
        train_loss.append(loss.data.cpu().numpy())
        
    print("Training loss (in-iteration): \t{:.6f}".format(
        np.mean(train_loss[-len(dataset) // batch_size :]))
    )

Epoch: 0
Logits shape: torch.Size([400, 7])
tensor(1.0000, grad_fn=<SumBackward0>)
tensor(51.5924, grad_fn=<SumBackward0>)
Gamma shape: torch.Size([400, 7])
N shape: torch.Size([1, 7])
tensor([[54.5114, 55.0887, 57.4135, 53.4241, 55.4211, 70.6198, 53.5214]],
       grad_fn=<SumBackward2>)
Pi shape: torch.Size([1, 7])
tensor([[0.1363, 0.1377, 0.1435, 0.1336, 0.1386, 0.1765, 0.1338]],
       grad_fn=<DivBackward0>)
Psi shape: torch.Size([7, 512])
Sigma shape: torch.Size([7, 512])


AttributeError: 'NoneType' object has no attribute 'backward'